In [2]:
# Description : This program predict of the stock price of a company will increase or decrease base on top news headlines.

In [115]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [168]:
# Import the Libraries
from textblob import TextBlob
from wordcloud import WordCloud 
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [169]:
# Load the dataset
df_1 = pd.read_csv("./data/data/Combined_News_DJIA.csv")
df_2 = pd.read_csv("./data/data/RedditNews.csv")
df_3 = pd.read_csv("./data/data/upload_DJIA_table.csv")

In [170]:
df_1.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."


In [171]:
df_2.head(2)

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host


In [172]:
df_3.head(2)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234


### Get dataset information

In [173]:
# shape of stock data
df_1.shape

(1989, 27)

In [174]:
# shape of news data
df_2.shape

(73608, 2)

In [175]:
df_3.shape

(1989, 7)

In [176]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1989 non-null   object
 1   Label   1989 non-null   int64 
 2   Top1    1989 non-null   object
 3   Top2    1989 non-null   object
 4   Top3    1989 non-null   object
 5   Top4    1989 non-null   object
 6   Top5    1989 non-null   object
 7   Top6    1989 non-null   object
 8   Top7    1989 non-null   object
 9   Top8    1989 non-null   object
 10  Top9    1989 non-null   object
 11  Top10   1989 non-null   object
 12  Top11   1989 non-null   object
 13  Top12   1989 non-null   object
 14  Top13   1989 non-null   object
 15  Top14   1989 non-null   object
 16  Top15   1989 non-null   object
 17  Top16   1989 non-null   object
 18  Top17   1989 non-null   object
 19  Top18   1989 non-null   object
 20  Top19   1989 non-null   object
 21  Top20   1989 non-null   object
 22  Top21   1989 non-null   

In [177]:
df_2.info

<bound method DataFrame.info of              Date                                               News
0      2016-07-01  A 117-year-old woman in Mexico City finally re...
1      2016-07-01   IMF chief backs Athens as permanent Olympic host
2      2016-07-01  The president of France says if Brexit won, so...
3      2016-07-01  British Man Who Must Give Police 24 Hours' Not...
4      2016-07-01  100+ Nobel laureates urge Greenpeace to stop o...
...           ...                                                ...
73603  2008-06-08  b'Man goes berzerk in Akihabara and stabs ever...
73604  2008-06-08  b'Threat of world AIDS pandemic among heterose...
73605  2008-06-08  b'Angst in Ankara: Turkey Steers into a Danger...
73606  2008-06-08  b"UK: Identity cards 'could be used to spy on ...
73607  2008-06-08  b'Marriage, they said, was reduced to the stat...

[73608 rows x 2 columns]>

In [178]:
df_3.info

<bound method DataFrame.info of             Date          Open          High           Low         Close  \
0     2016-07-01  17924.240234  18002.380859  17916.910156  17949.369141   
1     2016-06-30  17712.759766  17930.609375  17711.800781  17929.990234   
2     2016-06-29  17456.019531  17704.509766  17456.019531  17694.679688   
3     2016-06-28  17190.509766  17409.720703  17190.509766  17409.720703   
4     2016-06-27  17355.210938  17355.210938  17063.080078  17140.240234   
...          ...           ...           ...           ...           ...   
1984  2008-08-14  11532.070312  11718.280273  11450.889648  11615.929688   
1985  2008-08-13  11632.809570  11633.780273  11453.339844  11532.959961   
1986  2008-08-12  11781.700195  11782.349609  11601.519531  11642.469727   
1987  2008-08-11  11729.669922  11867.110352  11675.530273  11782.349609   
1988  2008-08-08  11432.089844  11759.959961  11388.040039  11734.320312   

         Volume     Adj Close  
0      82160000  17949.

In [179]:
# Merge the data set on data field
merge = df_1.merge(df_3, how='inner', on='Date', left_index=True)

In [180]:
# Show the merge data
merge.head(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
1988,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1987,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
1986,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727


In [181]:
# Combined the top news headlines
headlines = []

for row in range(0, len(merge.index)):
    headlines.append(' '.join(str(x) for x in merge.iloc[row, 2:27]))

In [182]:
# print a simple of the combined headlines
headlines[0]

'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\'

In [260]:
#remove punctuation
#apart from alphabets we will replace every character with blank
data = merge.iloc[:, 2:27]
#data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)

In [263]:
#renaming column
list1 = [i for i in range(25)]
new_Index = [str(i) for i in list1]
data.columns = new_Index
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
1988,b Georgia downs two Russian warplanes as cou...,b BREAKING Musharraf to be impeached,b Russia Today Columns of troops roll into So...,b Russian tanks are moving towards the capital...,b Afghan children raped with impunity U N ...,b Russian tanks have entered South Ossetia...,b Breaking Georgia invades South Ossetia Rus...,b The enemy combatent trials are nothing but...,b Georgian troops retreat from S Osettain cap...,b Did the U S Prep Georgia for War with Russia,...,b Georgia Invades South Ossetia if Russia ge...,b Al Qaeda Faces Islamist Backlash,b Condoleezza Rice The US would not act to p...,b This is a busy day The European Union has ...,b Georgia will withdraw soldiers from Ir...,b Why the Pentagon Thinks Attacking Iran is a ...,b Caucasus in crisis Georgia invades South Os...,b Indian shoe manufactory And again in a se...,b Visitors Suffering from Mental Illnesses Ban...,b No Help for Mexico s Kidnapping Surge
1987,b Why wont America and Nato help us If they w...,b Bush puts foot down on Georgian conflict,b Jewish Georgian minister Thanks to Israeli ...,b Georgian army flees in disarray as Russians ...,b Olympic opening ceremony fireworks faked,b What were the Mossad with fraudulent New Zea...,b Russia angered by Israeli military sale to G...,b An American citizen living in S Ossetia blam...,b Welcome To World War IV Now In High Definit...,b Georgia s move a mistake of monumental prop...,...,b Israel and the US behind the Georgian aggres...,b Do not believe TV neither Russian nor Geor...,b Riots are still going on in Montreal Canada...,b China to overtake US as largest manufacturer,b War in South Ossetia PICS,b Israeli Physicians Group Condemns State Tort...,b Russia has just beaten the United States ov...,b Perhaps the question about the Georgia R...,b Russia is so much better at war,b So this is what it s come to trading sex fo...
1986,b Remember that adorable year old who sang a...,b Russia ends Georgia operation,b If we had no sexual harassment we would hav...,b Al Qa eda is losing support in Iraq because ...,b Ceasefire in Georgia Putin Outmaneuvers the...,b Why Microsoft and Intel tried to kill the XO...,b Stratfor The Russo Georgian War and the Bal...,b I m Trying to Get a Sense of This Whole Geor...,b The US military was surprised by the timing ...,b U S Beats War Drum as Iran Dumps the Dollar,...,b U S troops still in Georgia did you know t...,b Why Russias response to Georgia was right,b Gorbachev accuses U S of making a serious ...,b Russia Georgia and NATO Cold War Two,b Remember that adorable year old who led y...,b War in Georgia The Israeli connection,b All signs point to the US encouraging Georgi...,b Christopher King argues that the US and NATO...,b America The New Mexico,b BBC NEWS Asia Pacific Extinction by man...
1985,b U S refuses Israel weapons to attack Iran ...,b When the president ordered to attack Tskhinv...,b Israel clears troops who killed Reuters cam...,b Britain s policy of being tough on drugs is...,b Body of year old found in trunk Latest ...,b China has moved million quake survivors...,b Bush announces Operation Get All Up In Russi...,b Russian forces sink Georgian ships,b The commander of a Navy air reconnaissance s...,b of CNN readers Russia s actions in Geor...,...,b Elephants extinct by,b US humanitarian missions soon in Georgia i...,b Georgia s DDOS came from US sources,b Russian convoy heads into Georgia violating...,b Israeli defence minister US against strike ...,b Gorbachev We Had No Choice,b Witness Russian forces head towards Tbilisi...,b Quarter of Russians blame U S for conflict...,b Georgian president says US military will ta...,b Nobel laureate Aleksander Solzhenitsyn...
1984,b All the experts admit that we should legalis...,b War in South Osetia pictures made by a ...,b Swedish wrestler Ara Abrahamian throws away ...,b Russia exaggerated the death toll in South O...,b Missile That Killed Inside Pakistan May Ha...

In [264]:
#converting hadlines to lower case
for index in new_Index:
    data[index] = data[index].str.lower()
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
1988,b georgia downs two russian warplanes as cou...,b breaking musharraf to be impeached,b russia today columns of troops roll into so...,b russian tanks are moving towards the capital...,b afghan children raped with impunity u n ...,b russian tanks have entered south ossetia...,b breaking georgia invades south ossetia rus...,b the enemy combatent trials are nothing but...,b georgian troops retreat from s osettain cap...,b did the u s prep georgia for war with russia,...,b georgia invades south ossetia if russia ge...,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to p...,b this is a busy day the european union has ...,b georgia will withdraw soldiers from ir...,b why the pentagon thinks attacking iran is a ...,b caucasus in crisis georgia invades south os...,b indian shoe manufactory and again in a se...,b visitors suffering from mental illnesses ban...,b no help for mexico s kidnapping surge
1987,b why wont america and nato help us if they w...,b bush puts foot down on georgian conflict,b jewish georgian minister thanks to israeli ...,b georgian army flees in disarray as russians ...,b olympic opening ceremony fireworks faked,b what were the mossad with fraudulent new zea...,b russia angered by israeli military sale to g...,b an american citizen living in s ossetia blam...,b welcome to world war iv now in high definit...,b georgia s move a mistake of monumental prop...,...,b israel and the us behind the georgian aggres...,b do not believe tv neither russian nor geor...,b riots are still going on in montreal canada...,b china to overtake us as largest manufacturer,b war in south ossetia pics,b israeli physicians group condemns state tort...,b russia has just beaten the united states ov...,b perhaps the question about the georgia r...,b russia is so much better at war,b so this is what it s come to trading sex fo...


In [265]:
# Combine all the 25 headlines to a single sentence

' '.join(str(x) for x in data.iloc[1, 0:25])  #example

'b why wont america and nato help us  if they wont help us now  why did we help them in iraq   b bush puts foot down on georgian conflict  b jewish georgian minister  thanks to israeli training  we re fending off russia   b georgian army flees in disarray as russians advance   gori abandoned to russia without a shot fired  b olympic opening ceremony fireworks  faked   b what were the mossad with fraudulent new zealand passports doing in iraq   b russia angered by israeli military sale to georgia  b an american citizen living in s ossetia blames u s  and georgian leaders for the genocide of innocent people  b welcome to world war iv  now in high definition   b georgia s move  a mistake of monumental proportions   b russia presses deeper into georgia  u s  says regime change is goal  b abhinav bindra wins first ever individual olympic gold medal for india  b  u s  ship heads for arctic to define territory  b drivers in a jerusalem taxi station threaten to quit rather than work for their 

In [267]:
#all the sentences in the form of list

headlines = []
for row in range(0, len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25]))


In [255]:
# Clean the data 
# clean_headlines = []

# for i in range(0, len(headlines)):
    
#     clean_headlines.append(re.sub("b[(')]", '', headlines[i]))           # remove 'b'
#     clean_headlines[i] = re.sub('b[(")]', '', clean_headlines[i])        # remove 'b'    
#     clean_headlines[i] = re.sub("\'", '', clean_headlines[i])            # remove \''

In [268]:
# Add the cleaned headlines to merge data set
merge['Combined_News'] = headlines

# Show the new columns
merge['Combined_News'][0]

'a     year old woman in mexico city finally received her birth certificate  and died a few hours later  trinidad alvarez lira had waited years for proof that she had been born in       imf chief backs athens as permanent olympic host the president of france says if brexit won  so can donald trump british man who must give police    hours  notice of sex threatens hunger strike  the man is the subject of a sexual risk order despite having never been convicted of a crime       nobel laureates urge greenpeace to stop opposing gmos brazil  huge spike in number of police killings in rio ahead of olympics austria s highest court annuls presidential election narrowly lost by right wing candidate  facebook wins privacy case  can track any belgian it wants  doesn t matter if internet users are logged into facebook or not switzerland denies muslim girls citizenship after they refuse to swim with boys at school  the     and    year old will no longer be considered for naturalised citizenship beca

In [269]:
# Create the function to get the subjectivity
def get_Subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def get_Polarity(text):
    return TextBlob(text).sentiment.polarity

In [270]:
# Create two columns 'Subjectivity' and polarity
merge['Subjecttivity'] = merge['Combined_News'].apply(get_Subjectivity)
merge['Polarity'] = merge['Combined_News'].apply(get_Polarity)

In [271]:
# Show the new column in the merge data set
merge.head(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Close,Volume,Adj Close,Combined_News,Subjecttivity,Polarity,Compound,Negative,Neutral,Positive
1988,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,11734.320312,212830000,11734.320312,b georgia downs two russian warplanes as cou...,0.267549,-0.048568,-0.9983,0.212,0.729,0.059
1987,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,11782.349609,183190000,11782.349609,b why wont america and nato help us if they w...,0.374806,0.109325,-0.9977,0.202,0.738,0.061
1986,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,11642.469727,173590000,11642.469727,b remember that adorable year old who sang a...,0.495171,-0.035676,-0.9975,0.225,0.684,0.091


## Stock Analysis using LinearDiscriminantAnalysis And vaderSentiment API

In [272]:
# Create a function to get the sentiment scores
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    
    Sentiment = sia.polarity_scores(text)
    return Sentiment

In [273]:
# Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = []

for i in range(0, len(merge['Combined_News'])):
    SIA = getSIA(merge['Combined_News'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])

In [274]:
# Store the Sentiment datasets
merge['Compound'] = compound
merge['Negative'] = neg
merge['Neutral'] = neu
merge['Positive'] = pos

In [275]:
merge.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Close,Volume,Adj Close,Combined_News,Subjecttivity,Polarity,Compound,Negative,Neutral,Positive
1988,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,11734.320312,212830000,11734.320312,b georgia downs two russian warplanes as cou...,0.267549,-0.048568,-0.9982,0.208,0.727,0.065
1987,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,11782.349609,183190000,11782.349609,b why wont america and nato help us if they w...,0.374806,0.109325,-0.9978,0.207,0.732,0.061
1986,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,11642.469727,173590000,11642.469727,b remember that adorable year old who sang a...,0.495171,-0.035676,-0.9979,0.238,0.668,0.094
1985,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,11532.959961,182550000,11532.959961,b u s refuses israel weapons to attack iran ...,0.364021,0.005842,-0.9571,0.133,0.765,0.103
1984,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,11615.929688,159790000,11615.929688,b all the experts admit that we should legalis...,0.390095,0.049050,-0.9618,0.144,0.760,0.096


In [276]:
    # Create a list of column to keep
keep_columns = ['Open', 'High', 'Low', 'Volume', 'Subjecttivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive', 'Neutral', 'Label']
df = merge[keep_columns]

In [277]:
df.head()

,Open,High,Low,Volume,Subjecttivity,Polarity,Compound,Negative,Neutral,Positive,Neutral,Label
1988,11432.089844,11759.959961,11388.040039,212830000,0.267549,-0.048568,-0.9982,0.208,0.727,0.065,0.727,0
1987,11729.669922,11867.110352,11675.530273,183190000,0.374806,0.109325,-0.9978,0.207,0.732,0.061,0.732,1
1986,11781.700195,11782.349609,11601.519531,173590000,0.495171,-0.035676,-0.9979,0.238,0.668,0.094,0.668,0
1985,11632.809570,11633.780273,11453.339844,182550000,0.364021,0.005842,-0.9571,0.133,0.765,0.103,0.765,0
1984,11532.070312,11718.280273,11450.889648,159790000,0.390095,0.049050,-0.9618,0.144,0.760,0.096,0.760,1


In [278]:
# Create the features data set 
X = df
X = np.array(X.drop(['Label'], 1))

# Create the target data set
y = np.array(df['Label'])

In [279]:
# Split the data into 80% training and 20 test data sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [280]:
# TRain the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [281]:
randomClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
randomClassifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [282]:
# Show the model prediction
predictions = model.predict(x_test)
predictions

array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,

In [283]:
random_pred = randomClassifier.predict(x_test)
random_pred

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,

In [284]:
y_test

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,

In [285]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [286]:
# Show the model metrics
matrix = confusion_matrix(y_test, predictions)
print(matrix)
score = accuracy_score(y_test, predictions)
print(score)
print(classification_report(y_test, predictions))

[[151  42]
 [ 24 181]]
0.8341708542713567
              precision    recall  f1-score   support

           0       0.86      0.78      0.82       193
           1       0.81      0.88      0.85       205

    accuracy                           0.83       398
   macro avg       0.84      0.83      0.83       398
weighted avg       0.84      0.83      0.83       398



In [287]:
matrix = confusion_matrix(y_test, random_pred)
print(matrix)
score = accuracy_score(y_test, random_pred)
print(score)
report = classification_report(y_test, random_pred)
print(report)

[[ 81 112]
 [ 75 130]]
0.5301507537688442
              precision    recall  f1-score   support

           0       0.52      0.42      0.46       193
           1       0.54      0.63      0.58       205

    accuracy                           0.53       398
   macro avg       0.53      0.53      0.52       398
weighted avg       0.53      0.53      0.52       398



## Stock Analysis using CountVectorizer & Random Forest Classifer

In [288]:
merge['Combined_News'][0]

'a     year old woman in mexico city finally received her birth certificate  and died a few hours later  trinidad alvarez lira had waited years for proof that she had been born in       imf chief backs athens as permanent olympic host the president of france says if brexit won  so can donald trump british man who must give police    hours  notice of sex threatens hunger strike  the man is the subject of a sexual risk order despite having never been convicted of a crime       nobel laureates urge greenpeace to stop opposing gmos brazil  huge spike in number of police killings in rio ahead of olympics austria s highest court annuls presidential election narrowly lost by right wing candidate  facebook wins privacy case  can track any belgian it wants  doesn t matter if internet users are logged into facebook or not switzerland denies muslim girls citizenship after they refuse to swim with boys at school  the     and    year old will no longer be considered for naturalised citizenship beca

In [289]:
merge.columns

Index(['Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close',
       'Combined_News', 'Subjecttivity', 'Polarity', 'Compound', 'Negative',
       'Neutral', 'Positive'],
      dtype='object')

In [290]:
df_count = merge[['Label', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Combined_News']]
df_count

,Label,Open,High,Low,Close,Volume,Adj Close,Combined_News
1988,0,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,b georgia downs two russian warplanes as cou...
1987,1,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,b why wont america and nato help us if they w...
1986,0,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,b remember that adorable year old who sang a...
1985,0,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,b u s refuses israel weapons to attack iran ...
1984,1,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,b all the experts admit that we should legalis...
...,...,...,...,...,...,...,...,...
4,0,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,barclays and rbs shares suspended from trading...
3,1,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,scientists to australia if you want to ...
2,1,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,explosion at airport in istanbul yemeni former...
1,1,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,jamaica proposes marijuana dispensers for tour...


In [291]:
# Create the features data set 
X = df_count

# Create the target data set
y = np.array(df_count['Label'])

In [292]:
# Split the data into 80% training and 20 test data sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [293]:
# implememt BAG OF WORDS
countVector = CountVectorizer(ngram_range=(2,2))
#ngram(2,2) means it will combine the 2 words together and assign the value

trainDataset = countVector.fit_transform(x_train['Combined_News'])

In [294]:
trainDataset

<1591x356531 sparse matrix of type '<class 'numpy.int64'>'
	with 659668 stored elements in Compressed Sparse Row format>

In [295]:
randomClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
randomClassifier.fit(trainDataset, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [296]:
test_dataset = countVector.transform(x_test['Combined_News'])
pred_count = randomClassifier.predict(test_dataset)

In [297]:
matrix = confusion_matrix(y_test, pred_count)
print(matrix)
score = accuracy_score(y_test, pred_count)
print(score)
report = classification_report(y_test, pred_count)
print(report)

[[  5 188]
 [  8 197]]
0.507537688442211
              precision    recall  f1-score   support

           0       0.38      0.03      0.05       193
           1       0.51      0.96      0.67       205

    accuracy                           0.51       398
   macro avg       0.45      0.49      0.36       398
weighted avg       0.45      0.51      0.37       398



## Stock Analysis using TFIDF & Random Forest Classifer

In [298]:
# implememt BAG OF WORDS
tfidfVector = TfidfVectorizer(ngram_range=(2,2))
#ngram(2,2) means it will combine the 2 words together and assign the value

trainDataset = tfidfVector.fit_transform(x_train['Combined_News'])

In [299]:
# Initialize Random forest classifier for Tfidf transform-fit dataset.
randomClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
randomClassifier.fit(trainDataset, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [300]:
# Tranform test dataset and predict.
test_dataset = tfidfVector.transform(x_test['Combined_News'])
pred_tfidf = randomClassifier.predict(test_dataset)

In [301]:
matrix = confusion_matrix(y_test, pred_tfidf)
print(matrix)
score = accuracy_score(y_test, pred_tfidf)
print(score)
report = classification_report(y_test, pred_tfidf)
print(report)

[[ 37 156]
 [ 49 156]]
0.4849246231155779
              precision    recall  f1-score   support

           0       0.43      0.19      0.27       193
           1       0.50      0.76      0.60       205

    accuracy                           0.48       398
   macro avg       0.47      0.48      0.43       398
weighted avg       0.47      0.48      0.44       398

